In [1]:
import sys
sys.path.append("../src")
import paths
RESULTS = paths.results

/home/user/mnlp/notebooks/../src/paths.py


In [2]:
print(RESULTS)

/home/user/mnlp/notebooks/../results


In [20]:
import pickle
import os
import pandas as pd

files = [
    "bert_OOD_tokens.csv",
    "deberta_OOD_tokens.csv",
    "modernbert_OOD_tokens.csv",
    "bert_val_tokens.csv",
    "deberta_val_tokens.csv",
    "modernbert_val_tokens.csv",
    "bert_dev_tokens.csv",
    "deberta_dev_tokens.csv",
    "modernbert_dev_tokens.csv",

    "gptpreddev.pkl",
    "gptpredOOD.pkl",
    "Mistral-7B-Instruct-v0.1-dev.pkl",
    "Mistral-7B-Instruct-v0.3-dev.pkl",
    "Mistral-7B-Instruct-v0.1-OOD.pkl",
    "Mistral-7B-Instruct-v0.3-OOD.pkl",
    "punctpredOOD.pkl",
    "punctpreddev.pkl",
    "Qwen2-7B-Instruct-OOD.pkl",
    "Qwen2-7B-Instruct-dev.pkl",


]

# Dictionary to hold loaded data
data = {}

# Loop through and read files
for fname in files:
    path = RESULTS/ fname
    if fname.endswith(".csv"):
        data[fname] = pd.read_csv(path)["pred"]
    elif fname.endswith(".pkl"):
        with open(path, "rb") as f:
            data[fname] = pickle.load(f)[0] if fname not in ["punctpreddev.pkl", "punctpredOOD.pkl"] else pickle.load(f)

# Show what was loaded
for k, v in data.items():
    print(f"{k}: {type(v)}")
    if isinstance(v, pd.DataFrame):
        print(v.head(), "\n")

DATA = paths.data
tokens = pd.read_csv(DATA/"manzoni_dev_tokens.csv")["token"]


bert_OOD_tokens.csv: <class 'pandas.core.series.Series'>
deberta_OOD_tokens.csv: <class 'pandas.core.series.Series'>
modernbert_OOD_tokens.csv: <class 'pandas.core.series.Series'>
bert_val_tokens.csv: <class 'pandas.core.series.Series'>
deberta_val_tokens.csv: <class 'pandas.core.series.Series'>
modernbert_val_tokens.csv: <class 'pandas.core.series.Series'>
bert_dev_tokens.csv: <class 'pandas.core.series.Series'>
deberta_dev_tokens.csv: <class 'pandas.core.series.Series'>
modernbert_dev_tokens.csv: <class 'pandas.core.series.Series'>
gptpreddev.pkl: <class 'list'>
gptpredOOD.pkl: <class 'list'>
Mistral-7B-Instruct-v0.1-dev.pkl: <class 'list'>
Mistral-7B-Instruct-v0.3-dev.pkl: <class 'list'>
Mistral-7B-Instruct-v0.1-OOD.pkl: <class 'list'>
Mistral-7B-Instruct-v0.3-OOD.pkl: <class 'list'>
punctpredOOD.pkl: <class 'list'>
punctpreddev.pkl: <class 'list'>
Qwen2-7B-Instruct-OOD.pkl: <class 'list'>
Qwen2-7B-Instruct-dev.pkl: <class 'list'>


In [67]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score   

SUBMISSION = RESULTS/"submission" 
for key,value in data.items():
    group_hw = f"last_slice_of_pizza-sentence_split-hw3_{split}-{model}.csv"
    if key in ["punctpredOOD.pkl", "punctpreddev.pkl"]:
        gold = pd.read_csv(DATA/"manzoni_dev_tokens.csv")["label"] if "dev" in key else pd.read_csv(DATA/"OOD_test.csv")["label"]
    if key[-3:] == "csv":
        gold = pd.read_csv(RESULTS/key)["label"]
    else:
        gold = pd.read_csv(DATA/"manzoni_dev_tokens.csv")["label"] if "dev" in key else pd.read_csv(DATA/"OOD_test.csv")["label"]
    print(key)
    print("gold: ", len(gold))
    print("value: ", len(value))
    prec, rec, f1, _ = precision_recall_fscore_support(
        gold, value, labels=[1], average="binary", zero_division=0
    )
    acc = accuracy_score(gold, value)

    if "dev" in key or "OOD" in key:
        if "gpt" in key: 
            model = "Gpt_oss_20b"
        elif "bert" in key:
            model = key[:-15]
        else:
            model = key[:-8]
        
        if "dev" in key:
            split = "dev" 
        else:
            split = "ood"
        
        df = pd.DataFrame({"label":value})
        df.to_csv(open(SUBMISSION/(group_hw.format(split, model)), "wb"), index=False)

    

    print({"precision": float(prec), "recall": float(rec), "f1": float(f1), "accuracy": float(acc)})

bert_OOD_tokens.csv
gold:  1725
value:  1725
{'precision': 0.7982456140350878, 'recall': 0.8198198198198198, 'f1': 0.8088888888888889, 'accuracy': 0.975072463768116}
deberta_OOD_tokens.csv
gold:  1698
value:  1698
{'precision': 0.9207920792079208, 'recall': 0.8857142857142857, 'f1': 0.9029126213592233, 'accuracy': 0.9882214369846879}
modernbert_OOD_tokens.csv
gold:  1554
value:  1554
{'precision': 0.8375, 'recall': 0.6907216494845361, 'f1': 0.7570621468926554, 'accuracy': 0.9723294723294723}
bert_val_tokens.csv
gold:  17274
value:  17274
{'precision': 0.9813559322033898, 'recall': 0.9914383561643836, 'f1': 0.9863713798977853, 'accuracy': 0.999073752460345}
deberta_val_tokens.csv
gold:  17305
value:  17305
{'precision': 0.998165137614679, 'recall': 0.9927007299270073, 'f1': 0.9954254345837146, 'accuracy': 0.999711066165848}
modernbert_val_tokens.csv
gold:  16499
value:  16499
{'precision': 0.9601990049751243, 'recall': 0.9523026315789473, 'f1': 0.9562345169281585, 'accuracy': 0.99678768

In [32]:
print(any([False if x in [0,1] else True for x in data["bert_OOD_tokens.csv"]]))

False


In [63]:
group_hw

'last_slice_of_pizza-sentence_split-hw3_dev-Qwen2-7B-Instruct.csv'

In [ ]:
value
df

,label
0,0
1,0
2,0
3,0
4,0
...,...
9338,0
9339,0
9340,0
9341,0
